<img align="right" src="tf-small.png"/>

# Ketiv Qere

This notebook can read ketiv-qere info in files issued by the etcbc and transform them 
into new features.
There will be new features at the word level.

**NB** This conversion will not work for versions `4` and `4b`.

## Discussion
There are already `qere` and `qere_utf8` features in the MQL of the core data.
However, there are several problems with those:

* features that contain the after-word material, `qere_trailer` and `qere_trailer_utf8`
  are missing;
* if there is no qere, both features are filled with the mepty string.
  In this way we can make no distinction between a truly empty qere and the absence of a qere.

That is why we reconstruct ketiv and qere from special files that are used by the ETCBC.

In [1]:
import os,sys,re,collections
from tf.fabric import Fabric
from tf.transcription import Transcription
import utils

In [2]:
if 'SCRIPT' not in locals():
    SCRIPT = False
    FORCE = True
    CORE_NAME = 'bhsa'
    VERSION= 'c'
    CORE_MODULE ='core' 

def stop(good=False):
    if SCRIPT: sys.exit(0 if good else 1)

# Setting up the context: source file and target directories

The conversion is executed in an environment of directories, so that sources, temp files and
results are in convenient places and do not have to be shifted around.

In [3]:
module = CORE_MODULE
repoBase = os.path.expanduser('~/github/etcbc')
thisRepo = '{}/{}'.format(repoBase, CORE_NAME)

thisSource = '{}/source/{}'.format(thisRepo, VERSION)

thisTemp = '{}/_temp/{}'.format(thisRepo, VERSION)
thisSave = '{}/{}'.format(thisTemp, module)

thisTf = '{}/tf/{}'.format(thisRepo, VERSION)
thisDeliver = '{}/{}'.format(thisTf, module)

In [4]:
testFeature = 'qere_trailer'

# Test

Check whether this conversion is needed in the first place.
Only when run as a script.

In [5]:
if SCRIPT:
    (good, work) = utils.mustRun(None, '{}/.tf/{}.tfx'.format(thisDeliver, testFeature), force=FORCE)
    if not good: stop(good=False)
    if not work: stop(good=True)

# TF Settings

* a piece of metadata that will go into these features; the time will be added automatically
* new text formats for the `otext` feature of TF, based on lexical features.
  We select the version specific otext material, 
  falling back on a default if nothing appropriate has been specified in oText.
 
We do not do this for the older versions 4 and 4b.

In [6]:
provenanceMetadata = dict(
    dataset='BHSA',
    datasetName='Biblia Hebraica Stuttgartensia Amstelodamensis',
    author='Eep Talstra Centre for Bible and Computer',
    encoders='Constantijn Sikkel (QDF), and Dirk Roorda (TF)',
    website='https://shebanq.ancient-data.org',
    email='shebanq@ancient-data.org',
)

oText = {
    'c': '''
@fmt:text-orig-full={qere_utf8/g_word_utf8}{qere_trailer_utf8/trailer_utf8}
@fmt:text-orig-full-ketiv={g_word_utf8}{trailer_utf8}
@fmt:text-trans-full={qere/g_word}{qere_trailer/trailer}
@fmt:text-trans-full-ketiv={g_word}{trailer}''',
}

thisOtext = oText.get(VERSION, '')
otextInfo = dict(line[1:].split('=', 1) for line in thisOtext.strip('\n').split('\n'))

if thisOtext is '':
    utils.caption(0, 'No additional text formats provided') 
else:
    utils.caption(0, 'New text formats')
for x in sorted(otextInfo.items()):
    utils.caption(0, '{:<30} = "{}"'.format(*x))

|       0.00s New text formats
|       0.00s fmt:text-orig-full             = "{qere_utf8/g_word_utf8}{qere_trailer_utf8/trailer_utf8}"
|       0.00s fmt:text-orig-full-ketiv       = "{g_word_utf8}{trailer_utf8}"
|       0.00s fmt:text-trans-full            = "{qere/g_word}{qere_trailer/trailer}"
|       0.00s fmt:text-trans-full-ketiv      = "{g_word}{trailer}"


In [7]:
utils.caption(4, 'Load the existing TF dataset')
TF = Fabric(locations=thisTf, modules=module)
api = TF.load('label g_word g_cons trailer_utf8')
F = api.F
Fs = api.Fs
T = api.T
L = api.L

..............................................................................................
.       3.85s Load the existing TF dataset                                                   .
..............................................................................................
This is Text-Fabric 2.3.12
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
99 features found and 0 ignored
  0.00s loading features ...
   |     0.12s B g_cons               from /Users/dirk/github/etcbc/bhsa/tf/c/core
   |     0.14s B g_word               from /Users/dirk/github/etcbc/bhsa/tf/c/core
   |     0.10s B trailer_utf8    

# Verse labels
The ketiv-qere files deal with different verse labels.
We make a mapping between verse labels and nodes.

In [8]:
utils.caption(0, 'Mapping between verse labels and verse nodes')
nodeFromLabel = {}
for vs in F.otype.s('verse'):
    lab = F.label.v(vs)
    nodeFromLabel[lab] = vs
utils.caption(0, '{} verses'.format(len(nodeFromLabel)))

|       9.99s Mapping between verse labels and verse nodes
|         10s 23213 verses


# Read the Ketiv-Qere file

In [9]:
utils.caption(4, 'Parsing Ketiv-Qere data')

verseInfo = collections.defaultdict(lambda: [])
notFound = set()
missing = collections.defaultdict(lambda: [])
missed = collections.defaultdict(lambda: [])

error_limit = 10

kqFile = '{}/ketivqere.txt'.format(thisSource)
kqHandle = open(kqFile)

ln = 0
can = 0
cur_label = None
for line in kqHandle:
    ln += 1
    can += 1
    vlab = line[0:10]
    fields = line.rstrip('\n')[10:].split()
    (ketiv, qere) = fields[0:2]
    (qtrim, qtrailer) = Transcription.suffix_and_finales(qere)
    vnode = nodeFromLabel.get(vlab, None)
    if vnode == None:
        notFound.add(vlab)
        continue
    verseInfo[vnode].append((ketiv, qtrim, qtrailer))        
kqHandle.close()
utils.caption(0, '\tRead {} ketiv-qere annotations'.format(ln))

..............................................................................................
.         13s Parsing Ketiv-Qere data                                                        .
..............................................................................................
|         13s 	Read 1892 ketiv-qere annotations


In [10]:
data = []

for vnode in verseInfo:
    wlookup = collections.defaultdict(lambda: [])
    wvisited = collections.defaultdict(lambda: -1)
    wnodes = L.d(vnode, otype='word')
    for w in wnodes:
        gw = F.g_word.v(w)
        if '*' in gw:
            gw = F.g_cons.v(w)
            if gw == '': gw = '.'
            if F.trailer_utf8.v(w) == '': gw += '-'
            wlookup[gw].append(w)
    for (ketiv, qere, qtrailer) in verseInfo[vnode]:
        wvisited[ketiv] += 1
        windex = wvisited[ketiv]
        ws = wlookup.get(ketiv, None)
        if ws == None or windex > len(ws) - 1:
            missing[vnode].append((windex, ketiv, qere))
            continue
        w = ws[windex]
        qereU = Transcription.to_hebrew(qere)
        qtrailerU = Transcription.to_hebrew(qtrailer)
        data.append((
            w,
            ketiv,
            qere,
            qtrailer.replace('\n', ''),
            qereU,
            qtrailerU.replace('\n', ''),
        ))
    for ketiv in wlookup:
        if ketiv not in wvisited or len(wlookup[ketiv]) - 1 > wvisited[ketiv]:
            missed[vnode].append((len(wlookup[ketiv]) - (wvisited.get(ketiv, -1) + 1), ketiv))
utils.caption(0, '\tParsed {} ketiv-qere annotations'.format(len(data)))

|         17s 	Parsed 1892 ketiv-qere annotations


In [ ]:
if not SCRIPT:
    print('\n'.join(repr(d) for d in data[0:10]))

In [11]:
if notFound:
    utils.caption(0, '\tWARNING: Could not find {} verses: {}'.format(len(notFound), sorted(notFound)))
else:
    utils.caption(0, '\tAll verses entries found in index')
if missing:
    utils.caption(0, '\tWARNING: Could not locate ketivs in the text: {} verses'.format(len(missing)))
    e = 0
    for vnode in sorted(missing):
        if e > error_limit: break
        vlab = F.label.v(vnode)
        for (windex, ketiv, qere) in missing[vnode]:
            e += 1
            if e > error_limit: break
            utils.caption(0, '\t\tNOT IN TEXT: {:<10} {:<20} #{} {}'.format(vlab, ketiv, windex, qere))
else:
    utils.caption(0, '\tAll ketivs found in the text')
if missed:
    utils.caption(0, '\tCould not lookup qeres in the data: {} verses'.format(len(missed)))
    e = 0
    for vnode in sorted(missed):
        if e > error_limit: break
        vlab = F.label.v(vnode)
        for (windex, ketiv) in missed[vnode]:
            e += 1
            if e > error_limit: break
            utils.caption(0, '\t\tNOT IN DATA: {:<10} {:<20} #{}'.format(vlab, ketiv, windex))
else:
    utils.caption(0, '\tAll ketivs found in the data')

|         22s 	All verses entries found in index
|         22s 	All ketivs found in the text
|         22s 	All ketivs found in the data


# Prepare TF features

We now collect the lexical information into the features for nodes of type `lex`.

In [12]:
utils.caption(0, 'Prepare TF ketiv qere features')

nodeFeatures = {}

newFeatures = '''
    qere
    qere_trailer
    qere_utf8
    qere_trailer_utf8
'''.strip().split()

nodeFeatures = dict( 
    qere=dict(((x[0], x[2]) for x in data)),
    qere_trailer=dict(((x[0], x[3]) for x in data)),
    qere_utf8=dict(((x[0], x[4]) for x in data)),
    qere_trailer_utf8=dict(((x[0], x[5]) for x in data))
)

|         37s Prepare TF ketiv qere features


We update the `otext` feature with new/changed formats

In [13]:
utils.caption(0, 'Update the otext feature')

metaData = {}

metaData['otext'] = dict()
metaData['otext'].update(T.config)
metaData['otext'].update(otextInfo)

for f in nodeFeatures:
    metaData[f] = {}
    metaData[f].update(provenanceMetadata)
    metaData[f]['valueType'] = 'str'

|         48s Update the otext feature


In [14]:
changedDataFeatures = set(nodeFeatures)
changedFeatures = changedDataFeatures | {'otext'}

# Stage: TF generation
Transform the collected information in feature-like datastructures, and write it all
out to `.tf` files.

In [15]:
utils.caption(4, 'write new/changed features to TF ...')
TF = Fabric(locations=thisSave, silent=True)
TF.save(nodeFeatures=nodeFeatures, edgeFeatures={}, metaData=metaData)

..............................................................................................
.         56s write new/changed features to TF ...                                           .
..............................................................................................
   |     0.01s T qere                 to /Users/dirk/github/etcbc/bhsa/_temp/c/core
   |     0.01s T qere_trailer         to /Users/dirk/github/etcbc/bhsa/_temp/c/core
   |     0.01s T qere_trailer_utf8    to /Users/dirk/github/etcbc/bhsa/_temp/c/core
   |     0.01s T qere_utf8            to /Users/dirk/github/etcbc/bhsa/_temp/c/core
   |     0.00s M otext                to /Users/dirk/github/etcbc/bhsa/_temp/c/core


# Stage: Diffs

Check differences with previous versions.

The new dataset has been created in a temporary directory,
and has not yet been copied to its destination.

Here is your opportunity to compare the newly created features with the older features.
You expect some differences in some features.

We check the differences between the previous version of the features and what has been generated.
We list features that will be added and deleted and changed.
For each changed feature we show the first line where the new feature differs from the old one.
We ignore changes in the metadata, because the timestamp in the metadata will always change.

In [36]:
utils.checkDiffs(thisSave, thisDeliver, only=changedFeatures)

..............................................................................................
.     37m 26s Check differences with previous version                                        .
..............................................................................................
|     37m 26s 	2 features to add
|     37m 26s 		qere_trailer
|     37m 26s 		qere_trailer_utf8
|     37m 26s 	no features to delete
|     37m 26s 	3 features in common
|     37m 26s otext                     ... no changes
|     37m 26s qere                      ... differencesafter the metadata
|     37m 27s 	line      2 OLD --><--
|     37m 27s 	line      2 NEW -->3897	HAJ:Y;74><--
|     37m 27s 	line      3 OLD --><--
|     37m 27s 	line      3 NEW -->4420	>@H:@LO75W<--
|     37m 27s 	line      4 OLD --><--
|     37m 27s 	line      4 NEW -->5645	>@H:@LO92W<--
|     37m 27s 	line      5 OLD --><--
|     37m 27s 	line      5 NEW -->5912	>@95H:@LOW03<--

|     37m 27s qere_utf8                 ... differe

# Stage: Deliver 

Copy the new TF dataset from the temporary location where it has been created to its final destination.

In [17]:
utils.deliverFeatures(thisSave, thisDeliver, changedFeatures)

..............................................................................................
.     16m 23s Deliver features to /Users/dirk/github/etcbc/bhsa/tf/c/core                    .
..............................................................................................
|     16m 23s 	qere_utf8
|     16m 23s 	qere
|     16m 23s 	otext
|     16m 23s 	qere_trailer_utf8
|     16m 23s 	qere_trailer


# Stage: Compile TF

We load the new features, use the new format, check some values

In [18]:
utils.caption(4, 'Load and compile the new TF features')

TF = Fabric(locations=thisTf, modules=module)
api = TF.load('g_word_utf8 g_word trailer_utf8 trailer {}'.format(' '.join(changedDataFeatures)))
F = api.F
Fs = api.Fs
T = api.T
L = api.L

..............................................................................................
.     17m 48s Load and compile the new TF features                                           .
..............................................................................................
This is Text-Fabric 2.3.12
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
101 features found and 0 ignored
  0.00s loading features ...
   |     0.14s B g_word               from /Users/dirk/github/etcbc/bhsa/tf/c/core
   |     0.21s B g_word_utf8          from /Users/dirk/github/etcbc/bhsa/tf/c/core
   |     0.01s T qere           

In [32]:
utils.caption(4, 'Basic tests')

def showKq(w):
    hw = F.g_word_utf8.v(w)
    tw = F.g_word.v(w)
    ht = F.trailer_utf8.v(w)
    tt = F.trailer.v(w)
    
    qhw = F.qere_utf8.v(w)
    qtw = F.qere.v(w)
    qht = F.qere_trailer_utf8.v(w)
    qtt = F.qere_trailer.v(w)
    
    utils.caption(0, '{:<20} {}'.format('hebrew', hw + ht))
    utils.caption(0, '{:<20} {}'.format('hebrew qere', qhw + qht))
    utils.caption(0, '{:<20} {}'.format('transcription', tw + tt))
    utils.caption(0, '{:<20} {}'.format('transcription qere', qtw + qtt))
    
utils.caption(0, '{:<30}: {}'.format(
    'absence of qere',
    ' '.join('NA' if F.qere.v(w) == None else F.qere.v(w) for w in (range(24700,24710))),
))
utils.caption(0, '{:<30}: {}'.format(
    'presence of qere trailer',
    ' '.join('NA' if F.qere_trailer.v(w) == None else F.qere_trailer.v(w) for w in (range(30190,30195))),
))

showNode = L.u(122073, otype='verse')[0]
showVerse = T.sectionFromNode(showNode)

utils.caption(4, '{} {}:{} in all formats'.format(*showVerse))
for fmt in T.formats:
    utils.caption(0, '{:<30} {}'.format(fmt, T.text(L.d(showNode, otype='word'), fmt=fmt)))

..............................................................................................
.     47m 48s Basic tests                                                                    .
..............................................................................................
|     47m 48s absence of qere               : NA WA J.I45C:T.AX:AW.75W. NA NA NA NA NA NA NA
|     47m 48s presence of qere trailer      : NA NA &   NA
..............................................................................................
.     47m 48s Joshua 15:53 in all formats                                                    .
..............................................................................................
|     47m 48s lex-orig-full                  ו ינים וּ בֵית תַּפּוּחַ וַ אֲפֵקָה 
|     47m 48s text-trans-full                W:J@N71W.m W.-B;JT&T.AP.73W.XA WA->:AP;75Q@H00 
|     47m 48s text-orig-full                 וְיָנ֥וּם וּבֵית־תַּפּ֖וּחַ וַאֲפֵֽקָה׃ 
|     47m 48s lex